In [ ]:
!pip install TTS

In [ ]:
import os

In [ ]:
# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

BaseDatasetConfig provides a configuration template for managing dataset-related settings in text-to-speech (TTS) applications

In [ ]:
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [ ]:
!wget -O tts_train_dir/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 

In [ ]:
!tar -xf tts_train_dir/LJSpeech-1.1.tar.bz2

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path="/content/LJSpeech-1.1")

In [ ]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1, #This value means that testing will be performed immediately without any delay after each epoch of training. The value -1 indicates that there is no specific number of epochs to wait before starting the testing phase.
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Note: A text cleaner is responsible for performing various text normalization operations to convert the raw input text into a cleaner representation suitable for TTS processing. The "phoneme_cleaners" text cleaner is likely designed to convert the input text into phonemes, which are the smallest units of sound in a language.

By using the "phoneme_cleaners" text cleaner, the TTS model will work with phoneme sequences instead of raw text, enabling it to generate speech that corresponds to the phonetic representation of the input text.

Now, we will initialize the audio processor using AudioProcessor class, responsible for handling feature extraction from audio and performing audio I/O operations.

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

Next we will initialize the tokenizer which is used to convert text to sequences of token IDs. If characters are not defined in the config, default characters are passed to the config.

In [ ]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

Next we will load data samples. Each sample is a list of [text, audio_file_path, speaker_name].

In [ ]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

In [ ]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [ ]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

In [ ]:
trainer.fit()


 > EPOCH: 0/100
 --> tts_train_dir/run-May-16-2023_05+33PM-0000000


[*] Pre-computing phonemes...


  0%|          | 4/12969 [00:00<37:50,  5.71it/s]

ɪnstɛd əv weɪtɪŋ ðɛɹ, ɔzwɔld əpɛɹəntli wɛnt æz fɑɹ əweɪ æz hi kʊd ænd bɔɹdɪd ðə fɚst oʊk klɪf bʌs wɪt͡ʃ keɪm əlɔŋ
 [!] Character '͡' not found in the vocabulary. Discarding it.


 16%|█▌        | 2059/12969 [01:20<05:06, 35.57it/s]

...


   --> STEP: 14/406 -- GLOBAL_STEP: 2450
     | > loss: 2.14259  (2.14212)
     | > log_mle: 0.52022  (0.53317)
     | > loss_dur: 1.62237  (1.60895)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 4.07272  (4.03119)
     | > current_lr: 0.00000 
     | > step_time: 0.50390  (0.72681)
     | > loader_time: 0.01850  (0.01334)


   --> STEP: 39/406 -- GLOBAL_STEP: 2475
     | > loss: 2.08823  (2.10643)
     | > log_mle: 0.52820  (0.53091)
     | > loss_dur: 1.56003  (1.57551)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.98090  (3.97942)
     | > current_lr: 0.00000 
     | > step_time: 0.49010  (0.71204)
     | > loader_time: 0.00890  (0.01240)


   --> STEP: 64/406 -- GLOBAL_STEP: 2500
     | > loss: 1.94730  (2.08417)
     | > log_mle: 0.52504  (0.52642)
     | > loss_dur: 1.42226  (1.55775)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.68343  (3.93746)
     | > current_lr: 0.00000 
     | > step_time: 0.65290  (0.74



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 3 not found characters:
	| > ͡
	| > “
	| > ”
| > Number of instances : 131
 | > Preprocessing samples
 | > Max text length: 174
 | > Min text length: 20
 | > Avg text length: 100.76335877862596
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 34739.0
 | > Avg audio length: 144033.41221374046
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.00996 (-0.00391)
     | > avg_loss: 1.72562 (-0.31746)
     | > avg_log_mle: 0.44169 (-0.07297)
     | > avg_loss_dur: 1.28393 (-0.24449)

 > BEST MODEL : tts_train_dir/run-May-16-2023_05+33PM-0000000/best_model_2842.pth

 > EPOCH: 7/100
 --> tts_train_dir/run-May-16-2023_05+33PM-0000000

 > TRAINING (2023-05-16 18:38:33) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
	| > 3 not found characters:
	| > ͡
	| > “
	| > ”
| > Number of instances : 12969
 | > Preprocessing samples
 | > Max text length: 188
 | > Min text length: 13
 | > Avg text length: 100.90014650319993
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 24499.0
 | > Avg audio length: 144984.29755570978
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.



   --> STEP: 8/406 -- GLOBAL_STEP: 2850
     | > loss: 1.81816  (1.82325)
     | > log_mle: 0.46040  (0.46742)
     | > loss_dur: 1.35776  (1.35583)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.15904  (3.20936)
     | > current_lr: 0.00000 
     | > step_time: 0.52400  (0.52279)
     | > loader_time: 0.00840  (0.00931)


   --> STEP: 33/406 -- GLOBAL_STEP: 2875
     | > loss: 1.79396  (1.78294)
     | > log_mle: 0.45811  (0.46281)
     | > loss_dur: 1.33585  (1.32012)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.19144  (3.16200)
     | > current_lr: 0.00000 
     | > step_time: 0.61600  (0.63241)
     | > loader_time: 0.00520  (0.00993)


   --> STEP: 58/406 -- GLOBAL_STEP: 2900
     | > loss: 1.79182  (1.76837)
     | > log_mle: 0.45139  (0.45966)
     | > loss_dur: 1.34043  (1.30871)
     | > amp_scaler: 16384.00000  (16384.00000)
     | > grad_norm: 3.14062  (3.12631)
     | > current_lr: 0.00000 
     | > step_time: 0.59670  (0.686

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

In [ ]:
!tts --text "Text for TTS" \
      --model_path $test_ckpt \
      --config_path $test_config \
      --out_path out.wav

In [ ]:
import IPython
IPython.display.Audio("out.wav")